# Lab 12: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Lab 9 imports
from funcs import *
from robotClass import *

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2022-05-06 21:42:38,318 | INFO     |: Logger demo_notebook.log initialized.


In [37]:
# Start the plotter
START_PLOTTER()

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [59]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble
        
        # Robot Control attribute
        #self.rc = RobotControl(ble)

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        self.rc.pingRobot(clear = True) # clear attributes
        sef.rc.turn360(forwardSpeed = 175, backwardSpeed = 175, dir_ = 0)
        await asyncio.sleep(10)
        
        sensorVals = np.array( [ val[0] / 1000 for val in self.rc.tof2_readings] ).reshape(18, 1)
        gyroVals = np.array( [ val[1] for val in self.rc.tof2_readings] ).reshape(18, 1)
        
        print(sensorVals)
        print(gyroVals)
        
        # RETURN DISTANCES AND ANGLES
        return sensorVals, gyroVals

In [22]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-05-06 21:59:44,336 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:B0:6C:9A:3C
2022-05-06 21:59:47,073 | INFO     |: Connected to C0:83:B0:6C:9A:3C


In [60]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-06 22:11:05,725 | INFO     |:  | Number of observations per grid cell: 18
2022-05-06 22:11:05,726 | INFO     |:  | Precaching Views...
2022-05-06 22:11:10,976 | INFO     |:  | Precaching Time: 5.248 secs
2022-05-06 22:11:10,977 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-06 22:11:10,978 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


### Lab 12 Testing

In [23]:
robot = RealRobot(cmdr, ble)

In [28]:
robot.rc.pingRobot(clear = True)
robot.rc.turn360(forwardSpeed = 175, backwardSpeed = 175, dir_ = 0)
await asyncio.sleep(10)

2022-05-06 22:01:10,431 | INFO     |: Disconnected from 607452C2-8441-D076-814E-841EB3635426


In [25]:
robot.rc.imu_readings

[(35.98482894897461, 1651888795.982444),
 (35.973548889160156, 1651888795.9993331),
 (55.05034255981445, 1651888797.1975548),
 (74.06098937988281, 1651888797.782754),
 (93.13629150390625, 1651888798.427572),
 (112.23027038574219, 1651888798.8326242),
 (131.26431274414062, 1651888799.3726442),
 (150.34181213378906, 1651888800.227513),
 (169.37391662597656, 1651888801.201819),
 (188.38470458984375, 1651888802.297791),
 (207.47802734375, 1651888803.017406),
 (226.5320587158203, 1651888803.43696),
 (245.56480407714844, 1651888803.9618628),
 (264.65179443359375, 1651888805.0135949),
 (283.689208984375, 1651888806.166949),
 (302.70318603515625, 1651888808.268267),
 (321.7370910644531, 1651888810.427228),
 (340.7401123046875, 1651888812.1821811)]

In [26]:
robot.rc.tof2_readings

[(409.0, 1651888795.997082),
 (412.0, 1651888797.212687),
 (237.0, 1651888797.811872),
 (273.0, 1651888798.42786),
 (225.0, 1651888798.84747),
 (220.0, 1651888799.373055),
 (265.0, 1651888800.242547),
 (282.0, 1651888801.217344),
 (329.0, 1651888802.298136),
 (350.0, 1651888803.032768),
 (432.0, 1651888803.451838),
 (389.0, 1651888803.97684),
 (595.0, 1651888805.0140648),
 (447.0, 1651888806.167201),
 (541.0, 1651888808.268658),
 (731.0, 1651888810.427509),
 (624.0, 1651888812.227804)]

# Run an update step of the Bayes Filter

In [61]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-06 22:11:13,736 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-06 22:11:13,737 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-05-06 22:11:13,739 | INFO     |: Update Step
2022-05-06 22:11:13,751 | INFO     |:      | Update Time: 0.009 secs
2022-05-06 22:11:13,755 | INFO     |: Bel index     : (0, 0, 0) with prob = nan
2022-05-06 22:11:13,757 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-06 22:11:13,760 | INFO     |: Belief        : (-1.524, -1.219, -170.000)
